In [1]:
import os, time
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path

#time_str = (datetime.datetime.now()).strftime("%Y%m%d")
time_str = '20190107'
xml_dir = '/data/darknet/python/' + time_str + '_xml/'
image_dir = '/data/darknet/python/' + time_str + '_image/'
generated_image_dir = '/data/darknet/python/' + time_str + '_image_fillter_obj/'
generated_xml_dir = '/data/darknet/python/' + time_str + '_xml_fillter_obj/'

os.system('mkdir -p ' + generated_xml_dir)
os.system('mkdir -p ' + generated_image_dir)

SEED_XML_DIR = xml_dir
SEED_IMG_DIR = image_dir
GENE_IMG_DIR = generated_image_dir
GENE_XML_DIR = generated_xml_dir
BKG_IMG_DIR = '/data/darknet/python/background/'

# RT:RightTop 
# LB:LeftBottom 
# bbox: [xmin, xax, ymin, ymax]
def IOU(bbox_a, bbox_b):
    '''
    W = min(A.RT.x, B.RT.x) - max(A.LB.x, B.LB.x) 
    H = min(A.RT.y, B.RT.y) - max(A.LB.y, B.LB.y) 
    if W <= 0 or H <= 0: 
        return 0 
    SA = (A.RT.x - A.LB.x) * (A.RT.y - A.LB.y) 
    SB = (B.RT.x - B.LB.x) * (B.RT.y - B.LB.y) 
    cross = W * H return cross/(SA + SB - cross)
    '''
    W = min(bbox_a[1], bbox_b[1]) - max(bbox_a[0], bbox_b[0]) 
    H = min(bbox_a[3], bbox_b[3]) - max(bbox_a[2], bbox_b[2]) 
    if W <= 0 or H <= 0: 
        return 0
    SA = (bbox_a[1] - bbox_a[0]) * (bbox_a[3] - bbox_a[2]) 
    SB = (bbox_b[1] - bbox_b[0]) * (bbox_b[3] - bbox_b[2])  
    cross = W * H 
    return cross/(SA + SB - cross)

def get_obj_from_xml(xml):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]

def get_obj_from_image_file(file, bbox):
    img = cv2.imread(file)
    img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
    return img_obj

def get_bboxes_from_etree(etree):
    root = tree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes
def past_to_background_from_image_file(file, bboxes, background_img_array, extend_spaces=0):
    
    img = cv2.imread(file)
    #img = img - 50
    if(img.shape != background_img_array.shape):
        print('shape not match')
        return
    #print(img.shape)
    #print(img)
    img_objs = []
    for bbox in bboxes:
        img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
        img_objs.append(img_obj)
    i = 0
    for bbox in bboxes:
        background_img_array[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])] = img_objs[i]
        i = i+1
    cv2.imwrite(GENE_IMG_DIR+file.split('/')[-1], background_img_array)
    return 

seed_xml_names = os.listdir(SEED_XML_DIR)
seed_xml_names.sort()
bkg_images = os.listdir(BKG_IMG_DIR)
bkg_images.sort()

classes_ = ['bus', 'car', 'truck', 'motorbike', 'bicycle', 'person']

last_frame_bboxes = []
for xml_name in seed_xml_names:
    #print(xml_name)       
    if(not xml_name.endswith('.xml')):
        continue
    
    #-------------------temp code start
    img_data = cv2.imread(BKG_IMG_DIR + bkg_images[3])
    #-------------------temp code end
    in_file = open(SEED_XML_DIR + xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    
    this_frame_bboxes = []
    boxes = []
    objs = []
    b = []
    #wrong_detection_objs = [[130, 311, 226, 475]]
    #objs_iter = root.iter('object')
    N = 0
    array_object = root.findall('object')
    for obj in array_object:
        erro_obj_removed = False
        stable_bbox_removed = False
        difficult = obj.find('difficult').text
        cls_ = obj.find('name').text
        if cls_ not in classes_ or int(difficult)==1:
            continue
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        this_frame_bboxes.append(b)
        if(len(last_frame_bboxes)>0):
            for stable_bbox in last_frame_bboxes:
                if(IOU(stable_bbox, b) > 0.85):
                    #print(b)
                    root.remove(obj)
                    stable_bbox_removed = True
                    N = N+1
                    break
            if(stable_bbox_removed):
                continue
        #filter size which smaller than 16X16 fit for SSD300
        bbox_width = b[1] - b[0]
        bbox_heigt = b[3] - b[2]
        if(not(bbox_width > 16 or bbox_heigt > 16)):
            print('remove:{0}:{1}'.format(bbox_width, bbox_heigt))
            root.remove(obj)
            continue 
        else:
            print('add:{0}:{1}'.format(bbox_width, bbox_heigt))
        boxes.append(b)
        objs += obj
    last_frame_bboxes = this_frame_bboxes.copy()  
    #print(boxes)
    
    if(len(boxes) == 0):
        continue
    past_to_background_from_image_file(SEED_IMG_DIR + xml_name[:-3] + 'jpg', boxes, img_data)
    tree.write(GENE_XML_DIR + xml_name)

add:28.0:69.0
add:46.0:130.0
add:77.0:135.0
add:59.0:46.0
add:73.0:131.0
add:59.0:85.0
add:73.0:120.0
add:29.0:78.0
add:41.0:64.0
add:33.0:79.0
add:31.0:50.0
add:66.0:47.0
add:115.0:46.0
add:33.0:81.0
add:129.0:62.0
add:148.0:48.0
add:111.0:47.0
add:103.0:130.0
add:31.0:94.0
add:116.0:47.0
add:79.0:60.0
add:33.0:101.0
add:65.0:118.0
add:201.0:27.0
add:57.0:126.0
add:38.0:104.0
add:66.0:130.0
add:184.0:36.0
add:66.0:67.0
add:100.0:150.0
add:101.0:119.0
add:101.0:119.0
add:134.0:46.0
add:102.0:148.0
add:127.0:112.0
add:39.0:59.0
add:94.0:50.0
add:38.0:102.0
add:107.0:144.0
add:122.0:124.0
add:99.0:153.0
add:18.0:45.0
add:73.0:122.0
add:126.0:116.0
add:38.0:38.0
add:41.0:61.0
add:43.0:105.0
add:90.0:129.0
add:134.0:120.0
add:42.0:102.0
add:119.0:104.0
add:84.0:122.0
add:125.0:96.0
add:47.0:119.0
add:66.0:118.0
add:87.0:118.0
add:40.0:120.0
add:60.0:108.0
add:119.0:88.0
add:57.0:110.0
add:43.0:121.0
add:57.0:118.0
add:48.0:114.0
add:61.0:107.0
add:48.0:56.0
add:52.0:113.0
add:57.0:104.0
ad

add:31.0:61.0
add:44.0:59.0
add:52.0:84.0
add:54.0:98.0
add:52.0:102.0
add:67.0:82.0
add:51.0:98.0
add:59.0:109.0
add:122.0:119.0
add:87.0:85.0
add:72.0:114.0
add:100.0:103.0
add:106.0:109.0
add:117.0:111.0
add:110.0:127.0
add:123.0:109.0
add:112.0:125.0
add:118.0:129.0
add:119.0:156.0
add:168.0:133.0
add:91.0:141.0
add:109.0:97.0
add:108.0:167.0
add:110.0:86.0
add:103.0:152.0
add:65.0:89.0
add:169.0:154.0
add:104.0:65.0
add:92.0:156.0
add:121.0:200.0
add:171.0:140.0
add:73.0:74.0
add:104.0:155.0
add:74.0:70.0
add:43.0:63.0
add:71.0:179.0
add:20.0:25.0
add:20.0:26.0
add:61.0:73.0
add:19.0:27.0
add:91.0:52.0
add:19.0:47.0
add:32.0:57.0
add:31.0:62.0
add:31.0:70.0
add:33.0:67.0
add:21.0:43.0
add:35.0:74.0
add:42.0:105.0
add:34.0:70.0
add:47.0:90.0
add:48.0:107.0
add:51.0:104.0
add:50.0:92.0
add:52.0:105.0
add:44.0:52.0
add:54.0:106.0
add:69.0:85.0
add:46.0:48.0
add:57.0:114.0
add:59.0:105.0
add:46.0:46.0
add:63.0:123.0
add:52.0:114.0
add:63.0:126.0
add:67.0:147.0
add:134.0:214.0
add:136.

add:36.0:88.0
add:37.0:96.0
add:116.0:107.0
add:35.0:55.0
add:37.0:83.0
add:45.0:103.0
add:33.0:47.0
add:36.0:95.0
add:41.0:110.0
add:133.0:35.0
add:33.0:95.0
add:21.0:33.0
add:40.0:106.0
add:39.0:103.0
add:23.0:30.0
add:133.0:44.0
add:38.0:102.0
add:115.0:45.0
add:16.0:27.0
add:41.0:115.0
add:13.0:23.0
add:86.0:50.0
add:12.0:18.0
add:44.0:119.0
remove:13.0:16.0
add:44.0:108.0
add:16.0:18.0
add:40.0:49.0
add:45.0:122.0
add:15.0:43.0
add:50.0:135.0
add:47.0:116.0
add:45.0:136.0
add:49.0:155.0
add:50.0:135.0
add:60.0:157.0
add:53.0:138.0
add:65.0:146.0
add:48.0:157.0
add:50.0:169.0
add:55.0:159.0
add:62.0:171.0
add:53.0:169.0
add:64.0:179.0
add:58.0:143.0
add:59.0:157.0
add:74.0:175.0
add:64.0:161.0
add:81.0:176.0
add:62.0:169.0
add:92.0:182.0
add:71.0:179.0
add:78.0:212.0
add:262.0:51.0
add:77.0:184.0
add:85.0:202.0
add:73.0:185.0
add:91.0:185.0
add:78.0:209.0
add:95.0:191.0
add:75.0:223.0
add:88.0:172.0
add:276.0:69.0
add:75.0:187.0
add:82.0:202.0
add:19.0:43.0
add:109.0:246.0
add:115.

add:37.0:82.0
add:37.0:49.0
add:37.0:53.0
add:34.0:83.0
add:32.0:53.0
add:34.0:69.0
add:49.0:122.0
add:32.0:66.0
add:26.0:98.0
add:51.0:127.0
add:32.0:63.0
add:67.0:129.0
add:28.0:36.0
add:43.0:121.0
add:33.0:58.0
add:115.0:99.0
add:37.0:41.0
add:58.0:127.0
add:64.0:76.0
add:43.0:126.0
add:121.0:124.0
add:27.0:37.0
add:127.0:149.0
add:46.0:113.0
add:76.0:70.0
add:58.0:130.0
add:122.0:44.0
add:101.0:160.0
add:114.0:60.0
add:119.0:150.0
add:28.0:35.0
add:48.0:134.0
add:91.0:62.0
add:44.0:110.0
add:110.0:139.0
add:26.0:31.0
add:104.0:148.0
add:63.0:70.0
add:98.0:142.0
add:44.0:102.0
add:60.0:113.0
add:33.0:56.0
add:74.0:222.0
add:80.0:150.0
add:45.0:99.0
add:55.0:123.0
add:75.0:139.0
add:124.0:197.0
add:47.0:102.0
add:129.0:194.0
add:75.0:141.0
add:60.0:81.0
add:43.0:103.0
add:118.0:218.0
add:43.0:105.0
add:69.0:106.0
add:53.0:96.0
add:25.0:74.0
add:46.0:114.0
add:64.0:119.0
add:122.0:227.0
add:38.0:85.0
add:26.0:99.0
add:42.0:111.0
add:36.0:91.0
add:56.0:129.0
add:49.0:106.0
add:36.0:88.

add:32.0:95.0
remove:8.0:7.0
add:48.0:106.0
add:30.0:96.0
remove:10.0:10.0
add:50.0:132.0
add:54.0:130.0
remove:12.0:12.0
add:52.0:130.0
remove:11.0:14.0
add:34.0:77.0
add:38.0:62.0
add:45.0:126.0
remove:13.0:16.0
add:55.0:119.0
add:62.0:109.0
add:52.0:118.0
add:115.0:118.0
add:12.0:19.0
add:29.0:81.0
add:129.0:130.0
add:48.0:117.0
add:151.0:123.0
add:50.0:116.0
add:152.0:136.0
add:168.0:119.0
add:104.0:141.0
add:47.0:112.0
add:29.0:79.0
add:180.0:128.0
add:45.0:107.0
add:27.0:79.0
add:44.0:102.0
add:31.0:70.0
add:30.0:71.0
add:105.0:140.0
add:33.0:90.0
add:44.0:57.0
add:31.0:87.0
add:28.0:69.0
add:28.0:64.0
add:18.0:45.0
add:32.0:83.0
add:138.0:116.0
add:17.0:49.0
add:116.0:152.0
add:15.0:52.0
add:33.0:80.0
add:26.0:67.0
add:27.0:57.0
add:123.0:209.0
add:33.0:78.0
add:23.0:52.0
add:79.0:52.0
add:113.0:231.0
add:25.0:55.0
add:31.0:74.0
add:109.0:232.0
add:28.0:53.0
add:24.0:60.0
add:24.0:61.0
add:117.0:237.0
add:25.0:61.0
add:92.0:225.0
add:85.0:208.0
add:28.0:65.0
add:71.0:195.0
add:3

...

add:43.0:119.0
add:51.0:129.0
add:30.0:60.0
add:55.0:55.0
add:27.0:115.0
add:16.0:21.0
add:32.0:36.0
add:47.0:125.0
add:39.0:111.0
add:30.0:52.0
add:57.0:56.0
add:21.0:30.0
add:46.0:124.0
add:30.0:115.0
add:27.0:48.0
add:58.0:58.0
add:15.0:25.0
add:17.0:26.0
add:20.0:30.0
add:50.0:115.0
add:37.0:101.0
add:67.0:63.0
add:22.0:38.0
add:15.0:26.0
add:32.0:30.0
add:33.0:92.0
add:41.0:110.0
add:22.0:37.0
add:19.0:30.0
add:15.0:27.0
add:62.0:55.0
add:18.0:24.0
add:32.0:91.0
add:14.0:29.0
add:12.0:29.0
add:20.0:33.0
add:34.0:91.0
add:69.0:65.0
add:21.0:18.0
add:27.0:90.0
add:29.0:95.0
add:22.0:40.0
add:13.0:32.0
remove:14.0:16.0
add:23.0:66.0
add:30.0:91.0
add:29.0:84.0
add:14.0:34.0
add:15.0:34.0
add:21.0:66.0
add:23.0:40.0
add:31.0:86.0
add:21.0:69.0
add:26.0:84.0
add:16.0:36.0
add:29.0:47.0
add:70.0:61.0
add:26.0:81.0
add:24.0:75.0
add:43.0:71.0
add:22.0:64.0
add:77.0:52.0
add:15.0:21.0
add:29.0:76.0
add:25.0:74.0
add:46.0:74.0
add:85.0:56.0
add:21.0:57.0
add:22.0:74.0
add:59.0:83.0
add:96.

add:46.0:106.0
add:33.0:86.0
add:42.0:86.0
add:43.0:106.0
add:28.0:94.0
add:34.0:95.0
add:45.0:115.0
add:30.0:89.0
add:36.0:99.0
add:47.0:118.0
add:47.0:105.0
add:28.0:94.0
add:57.0:127.0
add:28.0:101.0
add:40.0:120.0
add:53.0:135.0
add:61.0:115.0
add:31.0:101.0
add:63.0:159.0
add:55.0:129.0
add:41.0:112.0
add:66.0:156.0
add:53.0:130.0
add:45.0:103.0
add:65.0:168.0
add:69.0:144.0
add:52.0:152.0
add:70.0:196.0
add:35.0:93.0
add:88.0:205.0
add:76.0:167.0
add:69.0:187.0
add:103.0:203.0
add:106.0:214.0
add:112.0:172.0
add:111.0:114.0
add:155.0:118.0
add:95.0:98.0
add:68.0:38.0
add:39.0:64.0
add:55.0:52.0
add:68.0:54.0
add:59.0:83.0
add:92.0:75.0
add:48.0:72.0
add:18.0:21.0
add:19.0:24.0
add:28.0:25.0
add:19.0:26.0
add:6.0:25.0
add:13.0:25.0
add:7.0:20.0
add:30.0:21.0
add:12.0:20.0
add:12.0:25.0
add:15.0:24.0
add:13.0:26.0
add:35.0:25.0
add:21.0:34.0
add:35.0:27.0
add:15.0:25.0
add:9.0:23.0
add:13.0:26.0
add:11.0:27.0
add:41.0:24.0
add:15.0:27.0
add:13.0:29.0
add:16.0:23.0
add:16.0:23.0
add

add:48.0:115.0
add:46.0:120.0
add:77.0:164.0
add:33.0:71.0
add:38.0:55.0
add:52.0:116.0
add:33.0:74.0
add:48.0:135.0
add:70.0:160.0
add:32.0:98.0
add:44.0:123.0
add:79.0:189.0
add:33.0:83.0
add:48.0:107.0
add:52.0:125.0
add:89.0:203.0
add:43.0:86.0
add:54.0:108.0
add:53.0:122.0
add:90.0:231.0
add:41.0:80.0
add:60.0:149.0
add:48.0:104.0
add:62.0:141.0
add:47.0:136.0
add:104.0:232.0
add:56.0:91.0
add:53.0:127.0
add:61.0:153.0
add:38.0:90.0
add:107.0:219.0
add:38.0:89.0
add:72.0:157.0
add:47.0:132.0
add:28.0:96.0
add:41.0:90.0
add:94.0:199.0
add:76.0:167.0
add:62.0:110.0
add:48.0:127.0
add:120.0:178.0
add:86.0:167.0
add:47.0:109.0
add:50.0:139.0
add:22.0:36.0
add:21.0:32.0
add:51.0:108.0
add:86.0:178.0
add:55.0:138.0
add:25.0:54.0
add:112.0:194.0
add:62.0:123.0
add:30.0:68.0
add:54.0:119.0
add:68.0:135.0
add:61.0:97.0
add:35.0:65.0
add:57.0:167.0
add:57.0:123.0
add:31.0:71.0
add:41.0:76.0
add:28.0:59.0
add:32.0:88.0
add:31.0:83.0
add:18.0:19.0
add:17.0:13.0
add:35.0:29.0
add:17.0:13.0
add

add:17.0:36.0
add:15.0:37.0
add:15.0:35.0
add:15.0:37.0
add:21.0:44.0
add:13.0:31.0
add:20.0:30.0
add:17.0:38.0
add:15.0:34.0
add:13.0:32.0
add:16.0:39.0
add:16.0:41.0
add:19.0:29.0
add:25.0:49.0
add:19.0:28.0
add:17.0:39.0
add:16.0:39.0
add:16.0:40.0
add:13.0:30.0
add:16.0:41.0
add:25.0:55.0
add:17.0:41.0
add:18.0:42.0
add:16.0:44.0
add:18.0:48.0
add:26.0:59.0
add:15.0:36.0
add:20.0:49.0
add:17.0:40.0
add:17.0:42.0
add:17.0:40.0
add:24.0:69.0
add:20.0:31.0
add:17.0:42.0
add:18.0:43.0
add:17.0:50.0
add:27.0:55.0
add:17.0:35.0
add:17.0:44.0
add:18.0:45.0
add:16.0:51.0
add:24.0:56.0
add:17.0:45.0
add:24.0:62.0
add:18.0:51.0
add:17.0:37.0
add:19.0:46.0
add:25.0:64.0
add:26.0:51.0
add:23.0:54.0
add:22.0:39.0
add:18.0:47.0
add:19.0:50.0
add:24.0:57.0
add:24.0:63.0
add:22.0:38.0
add:18.0:49.0
add:27.0:70.0
add:26.0:62.0
add:18.0:53.0
add:26.0:50.0
add:17.0:50.0
add:23.0:56.0
add:27.0:69.0
add:27.0:62.0
add:40.0:84.0
add:18.0:45.0
add:16.0:57.0
add:29.0:59.0
add:22.0:57.0
add:23.0:75.0
add:29

add:28.0:60.0
add:35.0:71.0
add:26.0:69.0
add:29.0:68.0
add:27.0:72.0
add:31.0:72.0
add:31.0:75.0
add:34.0:80.0
add:35.0:82.0
add:37.0:85.0
add:45.0:104.0
add:37.0:96.0
add:49.0:108.0
